# Bollinger Bands Trading Strategy
This investment strategy is based on the Bollinger Band indicator. I'm using the <a href="https://vectorbt.dev">vector-bt library</a> for vectorized back-testing and parameter optimization. The goal of this notebook is to determine the optimal strategy parameters for three priceseries: BTCUSD, ETHUSD, and SOLUSD. 

In [1]:
from vectorbtpro import *

### Data Ingest 

In [2]:
data = vbt.HDFData.pull('priceseries.h5')

open_price = data.get('Open')
close_price = data.get('Close')
high_price = data.get('High')
low_price = data.get('Low')

### Indicator Parameter Space

In [3]:
wtypes = ["simple", "exp", "wilder"]
bbands_win = np.arange(10, 50, 1)
alphas = np.arange(1.5, 3.0, 0.05)

Bollinger-Band Calculation: $\alpha$ = number of standard deviations
$$
\begin{align*}
\text{Middle Band} &= \text{SMA}(\text{close}, n) \\
\text{Upper Band} &= \text{SMA}(\text{close}, n) + \alpha \times \sigma(\text{close}, n) \\
\text{Lower Band} &= \text{SMA}(\text{close}, n) - \alpha \times \sigma(\text{close}, n) \\\\
\text{Bandwidth} &= \frac{\text{Upper Band} - \text{Lower Band}}{\text{Middle Band}}
\end{align*}
$$

In [4]:
bbands = vbt.BBANDS.run(
    close_price,
    window=bbands_win,
    wtype=wtypes,
    alpha=alphas,
    param_product=True
)

### Trade Conditions

<u>Sell signal</u>: Downward breakout while expanding  
If the daily low is below the lower band *AND* the bandwidth is above the lower bandwidth threshold.

<u>Buy signal</u>: Upward breakout while squeezing  
If the daily high is above the upper band *AND* the bandwidth is below the upper bandwidth threshold.

In [5]:
lower_bandwidth_ths = 0.15
upper_bandwidth_ths = 0.3
bandwidth = (bbands.upper - bbands.lower) / bbands.middle

entry_cond1 = data.get("Low").vbt < bbands.lower
entry_cond2 = bandwidth > upper_bandwidth_ths
entry_cond3 = data.get("High").vbt > bbands.upper
entry_cond4 = bandwidth < lower_bandwidth_ths

in_cond1, in_cond2, in_cond3, in_cond4 = vbt.pd_acc.x(entry_cond1, entry_cond2, entry_cond3, entry_cond4)
entry_signals = (in_cond1.vbt & in_cond2).vbt | (in_cond3.vbt & in_cond4)

exit_cond1 = data.get("High").vbt > bbands.upper
exit_cond2 = bandwidth > 0.3
exit_cond3 = data.get("Low").vbt < bbands.lower
exit_cond4 = bandwidth < 0.15

out_cond1, out_cond2, out_cond3, out_cond4 = vbt.pd_acc.x(exit_cond1, exit_cond2, exit_cond3, exit_cond4)
exit_signals = (out_cond1.vbt & out_cond2).vbt | (out_cond3.vbt & out_cond4)

# Caclulate the maximum number of exit signals after each entry signal
exit_signals.vbt.signals.pos_rank_after(entry_signals, reset_wait=0).max() + 1

bb_window  bb_wtype  bb_alpha  symbol  
10         simple    1.50      X:BTCUSD    17
                               X:ETHUSD    14
                               X:SOLUSD    26
                     1.55      X:BTCUSD    16
                               X:ETHUSD    13
                                           ..
49         wilder    2.90      X:ETHUSD     1
                               X:SOLUSD     0
                     2.95      X:BTCUSD     3
                               X:ETHUSD     1
                               X:SOLUSD     0
Length: 10800, dtype: int64

#### Cleaning
Some signals shouln't be converted into orders. The maximum number of exit signals after each entry signal should be 1.

In [6]:
# select only the first signal (entry or exit), and ignore subsequent signals of the same type
entries, exits = entry_signals.vbt.signals.clean(exit_signals)

exits.vbt.signals.pos_rank_after(entries, reset_wait=0).max() + 1

bb_window  bb_wtype  bb_alpha  symbol  
10         simple    1.50      X:BTCUSD    1
                               X:ETHUSD    1
                               X:SOLUSD    1
                     1.55      X:BTCUSD    1
                               X:ETHUSD    1
                                          ..
49         wilder    2.90      X:ETHUSD    1
                               X:SOLUSD    0
                     2.95      X:BTCUSD    1
                               X:ETHUSD    1
                               X:SOLUSD    0
Length: 10800, dtype: int64

### Create a Portfolio from Entry and Exit Conditions

In [7]:
pf = vbt.Portfolio.from_signals(
    close=close_price, 
    entries=entries, 
    exits=exits,
    size=100,
    size_type='value',
    init_cash='auto'
)

performance_metrics = [
    'max_dd', 
    'max_dd_duration',
    'sortino_ratio',
    'omega_ratio',
    'profit_factor',
    'expectancy',
    'total_orders',
    'win_rate',
    'avg_winning_trade_duration',
    'avg_losing_trade_duration'
]

stats_df = pf.stats(performance_metrics, agg_func=None)

#### Analysis
Which parameter combination (Bollinger Band window and $\alpha$) yields the greatest Omega ratio (Risk-free rate $r_f = 0$)?
$$
\begin{align*}
\Omega(R, \tau) &= \frac{\int_{\tau}^{\infty} [1 - F(R)] \, dR}{\int_{-\infty}^{\tau} F(R) \, dR} \\\\
F(R) &= \text{CDF of returns} \\
\tau &= r_f \\
\end{align*}
$$


In [8]:
def get_wtype_stats(window_type, metric):
    win_mask = stats_df.index.get_level_values('bb_wtype') == window_type
    df = stats_df[win_mask]
    btc_mask = df.index.get_level_values('symbol') == 'X:BTCUSD'
    eth_mask = df.index.get_level_values('symbol') == 'X:ETHUSD'
    sol_mask = df.index.get_level_values('symbol') == 'X:SOLUSD'

    portfolio_data = pd.DataFrame({
        'BTCUSD': df[btc_mask][metric].values,
        'ETHUSD': df[eth_mask][metric].values,
        'SOLUSD': df[sol_mask][metric].values
    })

    return portfolio_data

In [12]:
pf_data = get_wtype_stats('exp', 'Omega Ratio')
param_comb = list(product(bbands_win, alphas))

pf_data.index = pd.MultiIndex.from_tuples(
    param_comb, 
    names=['bbands_win', 'bbands_alpha'])

btc_heatmap = pf_data['BTCUSD'].vbt.heatmap(x_level='bbands_alpha', y_level='bbands_win', return_fig=True)
btc_heatmap.update_layout(title="BTC/USD Heatmap (Ω Ratio)",
                         title_x = 0.5)

eth_heatmap = pf_data['ETHUSD'].vbt.heatmap(x_level='bbands_alpha', y_level='bbands_win', return_fig=True)
eth_heatmap.update_layout(title="ETH/USD Heatmap (Ω Ratio)",
                         title_x = 0.5)

sol_heatmap = pf_data['SOLUSD'].vbt.heatmap(x_level='bbands_alpha', y_level='bbands_win', return_fig=True)
sol_heatmap.update_layout(title="SOL/USD Heatmap (Ω Ratio)",
                         title_x = 0.5)

btc_heatmap.write_html("btc_heatmap.html", include_plotlyjs="embed")
eth_heatmap.write_html("eth_heatmap.html", include_plotlyjs="embed")
sol_heatmap.write_html("sol_heatmap.html", include_plotlyjs="embed")

<div style="display: flex; justify-content: center; align-items: center; width: 100%; height: 100%;">
    <iframe src="btc_heatmap.html" style="width: 100%; max-width: 1200px; height: 520px; border: none;" title="ETH Heatmap"></iframe>
</div>

<div style="display: flex; justify-content: center; align-items: center; width: 100%; height: 100%;">
    <iframe src="eth_heatmap.html" style="width: 100%; max-width: 1200px; height: 520px; border: none;" title="ETH Heatmap"></iframe>
</div>

<div style="display: flex; justify-content: center; align-items: center; width: 100%; height: 100%;">
    <iframe src="sol_heatmap.html" style="width: 100%; max-width: 1200px; height: 520px; border: none;" title="ETH Heatmap"></iframe>
</div>

For which asset did the BBANDS startegy perform the best based on omega ratios, win_rate, and expectancy?

In [10]:
df = stats_df[stats_df['Total Orders'] > 60]

omega_opt_index = df.sort_values(by='Omega Ratio', ascending=False).index[0]
win_rate_opt_index = df.sort_values(by='Win Rate [%]', ascending=False).index[0]
expectancy_opt_index = df.sort_values(by='Expectancy', ascending=False).index[0]

df.loc[[omega_opt_index, win_rate_opt_index, expectancy_opt_index]]

,,,,Max Drawdown [%],Max Drawdown Duration,Sortino Ratio,Omega Ratio,Profit Factor,Expectancy,Total Orders,Win Rate [%],Avg Winning Trade Duration,Avg Losing Trade Duration
bb_window,bb_wtype,bb_alpha,symbol,,,,,,,,,,
18,simple,1.60,X:ETHUSD,12.545993,453 days,3.281070,1.558652,4.637169,11.476915,93,50.000000,28 days 05:13:02.608695652,14 days 01:02:36.521739130
25,simple,1.60,X:ETHUSD,24.238010,355 days,2.147510,1.409932,4.643989,11.250759,63,64.516129,33 days 00:00:00,23 days 04:21:49.090909091
12,exp,1.55,X:ETHUSD,23.642496,694 days,2.045857,1.349401,4.025230,12.260608,71,54.285714,40 days 07:34:44.210526316,25 days 21:00:00


In [11]:
# Strategy parameters that yielded the portfolio with the highest expectancy
fig = pf[(12, "exp", 1.55, "X:ETHUSD")].plot() 
fig.write_html("figure2.html", include_plotlyjs="embed")

<iframe src="figure2.html" width="100%" height="1000"></iframe>